In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
file = '/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv'
df = pd.read_csv(file)
df.head()

# Data Exploration

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.isnull().any()

In [ ]:
df['quality'].value_counts()

In [ ]:
label = df['quality'].unique()
height = df['quality'].value_counts()
plt.bar(label,height)

# Searching for parameter correlation

I start by isolating the paramaters in my data in a matrice called X

In [ ]:
X = df.iloc[:,:-1]
X.head()

In [ ]:
sns.pairplot(X)

In [ ]:
corrmat = df.corr()
f, ax = plt.subplots(figsize =(9, 8)) 
sns.heatmap(corrmat, ax = ax, cmap ="YlGnBu", linewidths = 0.1) 

For the 'quality' factor which is the output we are trying to predict, there is no obvisous correlation between parameters so I decide to try to work on a smaller set of parameters for my mutltiparameter regression

# Multiparameter regression

I tried to work on a limited parameter dataset. I chose those that seemed to have the strongest correlation with 'wine quality' in the graphs above.

I call X_s my new parameter matrix

In [ ]:
X_s = X[['alcohol','sulphates']]

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_s = sc_X.fit_transform(X_s)

In [ ]:
y=df['quality']

In [ ]:
# Spliting the dataset between a train and a test set
from sklearn.model_selection import train_test_split
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_s,y, test_size = 0.2, random_state = 0)

In [ ]:
print(X_train_s.shape, y_train_s.shape)


In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train_s,y_train_s)

In [ ]:
reg.score(X_test_s,y_test_s)

This method doesn't give satisfactory results. A correlation factor of 0,28 is very low and suggest the model is not good a perdicting wine quality


# Classification method

In [ ]:
df_n = df

In [ ]:
df_n.shape

Let's create a new colomun called 'Good'. The values of this colomun are set to True when the wine is of quality >=6

In [ ]:
df_n['Good'] = df_n['quality']>=6

In [ ]:
df_n['Good'].sum()

In [ ]:
df_n.shape

In [ ]:
Y = pd.get_dummies(df_n['Good'], drop_first = True) ## Tip Use drop_first = True do avoid the dummies trap
Y.head()

In [ ]:
Y.sum()

In [ ]:
X = df_n[['fixed acidity','residual sugar','chlorides','free sulfur dioxide','density','pH','sulphates','alcohol']]

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X= sc_X.fit_transform(X)

In [ ]:
# Division du dataset en train et un test set 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
y_pred =  clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
print(classification_report(y_test,y_pred))


In [ ]:
print(accuracy_score(y_test,y_pred))

In [ ]:
print(confusion_matrix(y_test,y_pred))